In [1]:
%%capture --no-stderr
%pip install --quiet -U langchain_openai langchain_core langgraph langgraph-prebuilt langgraph_sdk langgraph-checkpoint-sqlite langsmith langchain-community tavily-python wikipedia

In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
import operator
from typing import List, Annotated, TypedDict
from langchain_core.messages import AnyMessage, HumanMessage
from langgraph.graph import StateGraph, END, START
from langgraph.graph.message import add_messages

# 1. Defining The State

class TravelAgentState(TypedDict):
    
    
    messages: Annotated[List[AnyMessage], add_messages] 
    
    
    destination: str
    budget: str
    travel_dates: str
    
    flight_options: Annotated[List[dict], operator.add]
    train_options: Annotated[List[dict], operator.add]
    bus_options: Annotated[List[dict], operator.add]
    accommodation_options: Annotated[List[dict], operator.add]


In [4]:
# 2. Setting Up Memory
import sqlite3
conn = sqlite3.connect(":memory:", check_same_thread = False)
from langgraph.checkpoint.sqlite import SqliteSaver
memory = SqliteSaver(conn)

In [5]:
from langchain_core.tools import tool
from langchain_openai import ChatOpenAI
from langgraph.prebuilt import ToolNode, tools_condition
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
import os

# 3. Defining The Tools
@tool
def search_flights(destination: str, dates: str, budget: str) -> List[dict]:
    """Searches for flights based on destination, dates, and budget."""
    print(f"--- TOOL: Searching flights to {destination} ---")
    return [{"type": "flight", "id": "fl_001", "name": "LangAir", "price": 450, "link": "https://example.com/flights/fl_001"}]
@tool
def search_trains(destination: str, dates: str) -> List[dict]:
    """Searches for trains based on destination and dates."""
    print(f"--- TOOL: Searching trains to {destination} ---")
    return [{"type": "train", "id": "trn_001", "name": "GraphRail", "price": 90, "link": "https://example.com/trains/trn_001"}]
@tool
def search_buses(destination: str, dates: str) -> List[dict]:
    """Searches for buses based on destination and dates."""
    print(f"--- TOOL: Searching buses to {destination} ---")
    return [{"type": "bus", "id": "bus_001", "name": "NodeExpress", "price": 50, "link": "https://example.com/buses/bus_001"}]
@tool
def search_hotels(destination: str, dates: str, budget: str) -> List[dict]:
    """Searches for hotels in the given destination for the given dates."""
    print(f"--- TOOL: Searching hotels in {destination} ---")
    return [{"type": "hotel", "id": "htl_001", "name": "The Checkpointer Inn", "price": 120, "link": "https://example.com/hotels/htl_001"}]
@tool
def search_airbnbs(destination: str, dates: str, budget: str) -> List[dict]:
    """Searches for Airbnbs in the given destination for the given dates."""
    print(f"--- TOOL: Searching Airbnbs in {destination} ---")
    return [{"type": "airbnb", "id": "ab_001", "name": "Cozy Loft by the Nodes", "price": 90, "link": "https://example.com/airbnb/ab_001"}]
tools = [search_flights, search_trains, search_buses, search_hotels, search_airbnbs]


# 4. Defining The Agent with System Message

llm = ChatOpenAI(model="gpt-4o-mini") 
llm_with_tools = llm.bind_tools(tools)

SYSTEM_MESSAGE = (
    "You are a helpful travel planning assistant. "
    "When a user provides their destination, dates, and budget, your goal is to find all available travel and accommodation options. "
    "When you have the results from your tools, present them clearly to the user in a list, including the type, name, price, and booking link. "
    "Also give them the total cost of their selected options. "
    "Do not make up any information that is not from your tools."
)



def assistant_node(state: TravelAgentState):
    """
    This is the main agent node. It calls the LLM, which can either
    respond directly to the user or decide to call one or more tools.
    """

    messages_with_system_prompt = [SystemMessage(content=SYSTEM_MESSAGE)] + state['messages']
    
    response = llm_with_tools.invoke(messages_with_system_prompt)
    
    return {"messages": [response]}

tool_node = ToolNode(tools)

In [6]:
# 5. Building the graph

builder = StateGraph(TravelAgentState)

builder.add_node("assistant", assistant_node)
builder.add_node("tools", tool_node)

builder.add_edge(START, "assistant")


builder.add_conditional_edges(
    "assistant",
    tools_condition,
    {
        "tools": "tools",
        END: END
    }
)

builder.add_edge("tools", "assistant")

app = builder.compile(checkpointer=memory)

In [7]:
from langgraph.graph import StateGraph, END, START
from langgraph.prebuilt import ToolNode
from langchain_core.messages import AIMessage, ToolMessage, SystemMessage
import json

# 6. Setting Up The Travel Specialist Sub-Graph
travel_tools = [search_flights, search_trains, search_buses]
travel_tool_node = ToolNode(travel_tools)

travel_model = ChatOpenAI(model="gpt-4o-mini").bind_tools(travel_tools)

travel_system_prompt = SystemMessage(content="""You are a travel specialist. 
Your task is to search for travel options (flights, trains, buses) using the available tools.
When a user asks for travel options, you MUST call the relevant search tools.
Do not ask clarifying questions. Just search.""")

# 7. Defining the Nodes

def travel_assistant_node(state: TravelAgentState):
    """
    The 'brain' of the Travel Specialist. 
    """
    messages = [travel_system_prompt] + state['messages']
    
    response = travel_model.invoke(messages)
    return {"messages": [response]}


def parse_travel_outputs(state: TravelAgentState):
    """
    The 'parser' node.
    Scans BACKWARDS through messages to find all tool outputs from this turn.
    """
    messages = state['messages']
    
    updates = {
        "flight_options": [],
        "train_options": [],
        "bus_options": [],
    }
    
    for msg in reversed(messages):
        if isinstance(msg, ToolMessage):
            try:
                data = eval(msg.content)
                
                if isinstance(data, list) and len(data) > 0:
                    item_type = data[0].get("type")
                    if item_type == "flight":
                        updates["flight_options"].extend(data)
                    elif item_type == "train":
                        updates["train_options"].extend(data)
                    elif item_type == "bus":
                        updates["bus_options"].extend(data)
            except Exception as e:
                print(f"Error parsing tool output: {e}")
        
        elif isinstance(msg, AIMessage):
            break
            
    return updates


# 8. Building the Travel Specialist Sub-Graph

travel_builder = StateGraph(TravelAgentState)

travel_builder.add_node("travel_assistant", travel_assistant_node)
travel_builder.add_node("travel_tools", travel_tool_node)
travel_builder.add_node("travel_parser", parse_travel_outputs)


travel_builder.add_edge(START, "travel_assistant")
travel_builder.add_edge("travel_assistant", "travel_tools")
travel_builder.add_edge("travel_tools", "travel_parser")
travel_builder.add_edge("travel_parser", END)

travel_agent_graph = travel_builder.compile()


In [8]:
# 9. Setting Up The Accommodation Specialist Sub-Graph
accom_tools = [search_hotels, search_airbnbs]
accom_tool_node = ToolNode(accom_tools)

accom_model = ChatOpenAI(model="gpt-4o-mini").bind_tools(accom_tools)

accom_system_prompt = SystemMessage(content="""You are an accommodation specialist. 
Your task is to search for hotels and Airbnbs using the available tools.
When a user asks for accommodation options, you MUST call the relevant search tools.
Do not ask clarifying questions. Just search.""")

# 10. Defining the Nodes

def accom_agent_node(state: TravelAgentState):
    """
    The 'brain' of the Accommodation Specialist. 
    """
    
    messages = [accom_system_prompt] + state['messages']
    
    response = accom_model.invoke(messages)
    return {"messages": [response]}

def parse_accom_outputs(state: TravelAgentState):
    """
    The 'parser' node for accommodation.
    Scans BACKWARDS through messages to find all tool outputs.
    """
    messages = state['messages']
    
    updates = {
        "accommodation_options": []
    }

    for msg in reversed(messages):
        if isinstance(msg, ToolMessage):
            try:
                data = eval(msg.content)
                if isinstance(data, list) and len(data) > 0:
                    item_type = data[0].get("type")
                    if item_type in ["hotel", "airbnb"]:
                        updates["accommodation_options"].extend(data)
            except Exception as e:
                print(f"Error parsing tool output: {e}")
        
        elif isinstance(msg, AIMessage):
            break
            
    return updates


# 11. Building the Accommodation Specialist Sub-Graph

accom_builder = StateGraph(TravelAgentState)

accom_builder.add_node("accom_assistant", accom_agent_node)
accom_builder.add_node("accom_tools", accom_tool_node)
accom_builder.add_node("accom_parser", parse_accom_outputs)

accom_builder.add_edge(START, "accom_assistant")
accom_builder.add_edge("accom_assistant", "accom_tools")
accom_builder.add_edge("accom_tools", "accom_parser")
accom_builder.add_edge("accom_parser", END)

accommodation_agent_graph = accom_builder.compile()

In [9]:
from langgraph.graph import StateGraph, END, START
from langgraph.types import Send
from langchain_core.messages import AIMessage, HumanMessage
import sqlite3
from langgraph.checkpoint.sqlite import SqliteSaver

# 12. Defining the Main Graph

def intake_node(state: TravelAgentState):
    """
    Parses the user's request to populate the state.
    """
    return {
        "destination": "Tokyo", 
        "dates": "2026-01-01 to 2026-01-07", 
        "budget": "$2000",
        "flight_options": [],
        "train_options": [],
        "bus_options": [],
        "accommodation_options": []
    }

def planner_node(state: TravelAgentState):
    """
    The Planner Node. Pass-through for logging.
    """
    return {}

def route_to_specialists(state: TravelAgentState):
    """
    The Conditional Edge Logic.
    Decides whether to trigger the parallel agents.
    """
    last_message = state['messages'][-1]
    
    if "plan" in last_message.content.lower() or "find" in last_message.content.lower():
        task_content = f"Find options for: {last_message.content}"
        task_message = HumanMessage(content=task_content)
        
        return [
            Send("travel_agent", {"messages": [task_message]}),
            Send("accommodation_agent", {"messages": [task_message]})
        ]
    
    return []

def present_plan_node(state: TravelAgentState):
    """
    REDUCE STEP: Aggregates all options into a final response.
    """
    print("--- MAIN GRAPH: Present Plan Node (Reduce) ---")
    
    flights = state.get("flight_options", [])
    trains = state.get("train_options", [])
    buses = state.get("bus_options", [])
    stays = state.get("accommodation_options", [])
    
    response_text = f"Here are the options I found for your trip to {state.get('destination', 'your destination')}:\n\n"
    
    def add_section(title, items):
        text = ""
        if items:
            text += f"**{title}:**\n"
            for i in items:
                text += f"- {i.get('name')} (${i.get('price')}): [Book Here]({i.get('link')})\n"
            text += "\n"
        return text

    response_text += add_section("Flights", flights)
    response_text += add_section("Trains", trains)
    response_text += add_section("Buses", buses)
    response_text += add_section("Accommodation", stays)
    
    if not (flights or trains or buses or stays):
        response_text = "I couldn't find any specific options. Please try refining your request."
    else:
        response_text += "You can click the links above to book these options directly!"

    return {"messages": [AIMessage(content=response_text)]}


# 13. Building the Main Graph

conn = sqlite3.connect("travel_agent.db", check_same_thread=False)
memory = SqliteSaver(conn=conn)

main_builder = StateGraph(TravelAgentState)

main_builder.add_node("intake", intake_node)
main_builder.add_node("planner", planner_node)
main_builder.add_node("present_plan", present_plan_node)

main_builder.add_node("travel_agent", travel_agent_graph)
main_builder.add_node("accommodation_agent", accommodation_agent_graph)

main_builder.add_edge(START, "intake")
main_builder.add_edge("intake", "planner")

main_builder.add_conditional_edges(
    "planner", 
    route_to_specialists,
    ["travel_agent", "accommodation_agent"] 
)

main_builder.add_edge("travel_agent", "present_plan")
main_builder.add_edge("accommodation_agent", "present_plan")

main_builder.add_edge("present_plan", END)


# 14. Compiling the Main Graph
app = main_builder.compile(checkpointer=memory)

In [10]:
# 15. Testing the Full Flow
config = {"configurable": {"thread_id": "final-demo-1"}}
user_input = {"messages": [HumanMessage(content="Plan a trip to Tokyo.")]}

for chunk in app.stream(user_input, config, stream_mode="values"):
    if "messages" in chunk:
        chunk['messages'][-1].pretty_print()

================================ Human Message =================================

Plan a trip to Tokyo.
================================ Human Message =================================

Plan a trip to Tokyo.
--- TOOL: Searching hotels in Tokyo ---
--- TOOL: Searching Airbnbs in Tokyo ---
--- TOOL: Searching flights to Tokyo ---
--- TOOL: Searching trains to Tokyo ---
--- TOOL: Searching buses to Tokyo ---
================================= Tool Message =================================
Name: search_airbnbs

[{"type": "airbnb", "id": "ab_001", "name": "Cozy Loft by the Nodes", "price": 90, "link": "https://example.com/airbnb/ab_001"}]
--- MAIN GRAPH: Present Plan Node (Reduce) ---
================================== Ai Message ==================================

Here are the options I found for your trip to Tokyo:

**Flights:**
- LangAir ($450): [Book Here](https://example.com/flights/fl_001)
- LangAir ($450): [Book Here](https://example.com/flights/fl_001)

**Trains:**
- GraphRail ($90):